## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,69,0,5.75,clear sky
1,1,Atasu,KZ,48.6814,71.6433,84.25,22,0,7.72,clear sky
2,2,Kapaa,US,22.0752,-159.3190,79.12,78,40,14.97,scattered clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,69.55,93,80,4.00,broken clouds
4,4,Namibe,AO,-15.1961,12.1522,66.88,76,0,5.23,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atasu,KZ,48.6814,71.6433,84.25,22,0,7.72,clear sky
2,2,Kapaa,US,22.0752,-159.3190,79.12,78,40,14.97,scattered clouds
5,5,Atuona,PF,-9.8000,-139.0333,77.04,68,22,15.70,few clouds
6,6,Saint-Pierre,RE,-21.3393,55.4781,76.91,66,0,20.71,clear sky
7,7,Pozhva,RU,59.0912,56.0847,77.38,50,79,4.97,broken clouds
...,...,...,...,...,...,...,...,...,...,...
661,661,At-Bashi,KG,41.1702,75.8106,84.65,16,0,12.91,clear sky
667,667,Kazanskaya,RU,45.4106,40.4361,83.52,37,13,11.34,few clouds
668,668,Tahe,CN,52.3333,124.7333,79.21,55,92,2.77,overcast clouds
671,671,Kadhan,PK,24.4810,68.9859,87.60,69,100,19.55,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

270


City_ID                270
City                   270
Country                270
Lat                    270
Lng                    270
Max Temp               270
Humidity               270
Cloudiness             270
Wind Speed             270
Current Description    270
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atasu,KZ,84.25,clear sky,48.6814,71.6433,
2,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,
5,Atuona,PF,77.04,few clouds,-9.8000,-139.0333,
6,Saint-Pierre,RE,76.91,clear sky,-21.3393,55.4781,
7,Pozhva,RU,77.38,broken clouds,59.0912,56.0847,
8,Joshimath,IN,75.65,overcast clouds,30.5667,79.5667,
11,Rikitea,PF,75.34,scattered clouds,-23.1203,-134.9692,
19,High Rock,BS,77.02,overcast clouds,26.6208,-78.2833,
20,Mahebourg,MU,75.25,scattered clouds,-20.4081,57.7000,
22,Bambous Virieux,MU,75.22,scattered clouds,-20.3428,57.7575,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atasu,KZ,84.25,clear sky,48.6814,71.6433,Dostyk
2,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,Atuona,PF,77.04,few clouds,-9.8000,-139.0333,Villa Enata
6,Saint-Pierre,RE,76.91,clear sky,-21.3393,55.4781,Alize Plage
8,Joshimath,IN,75.65,overcast clouds,30.5667,79.5667,Shivalik Camps & Resort


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))